In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
from tqdm import tqdm
import re
import sys

In [2]:
def open_excel():
    df = pd.read_excel(r'C:/Users/smhrd/food_info_.xlsx')
    
    # 필요없는 부분 -> Food_group, Food_mf 등등 삭제
    # 쓸 부분 - seq, name, recipe, weigth, calor, crb, protein, fat 씁니다.
    # 컬럼 제거
    df.drop(
        ['식품군','제조사명','지역명','당류(g)','나트륨(mg)','콜레스트롤(mg)'
        ,'포화지방산(g)','트랜스지방(g)'],
        axis=1,
        errors='ignore',
        inplace=True
    )
    
    # Food_name 전처리하기
    df['음식명'] = df['음식명'].apply(lambda x: re.sub('\s(?![a-z])|_|\((.*?)\)|(.*?)\?','', str(x)))
    df.head()
    
    # 중복 확인
    df.iloc[:,:7][df.duplicated(subset=['FOOD_SEQ','음식명'])]
    
    # 레시피가 nan값이 아닌 애들로만 구성.
    df = df[df['레시피'].notna()]
    # 인덱스 값 새롭게 지정하기
    df = df.reset_index(drop=True)
    return df

# 개인 정보에 기반한 활동 대사량, 기초대사량 구하기
# Physical Activity Level (PAL)

# *args로 하면 가변인자를 사용할 수 있다고 하네요!!
# def user_PAL(*args):
    
#     weight, height, age, gender, activity, purpose = args
def user_PAL(weight, height, age, gender, activity, purpose):
    # 기초대사량 -> 키, 몸무게, 나이로 구성된 기본식
    base = 10 * weight + 6.25 * height - 5 * age

    # 성별에 따른 계산
    if gender == 'm':
        man_base = base + 5
        crb, protein, fat = 0.5, 0.15, 0.35
        user = {'cal': man_base, 'crb': crb, 'protein': protein, 'fat': fat}
    else:
        woman_base = base - 161
        crb, protein, fat = 0.5, 0.15, 0.35
        user = {'cal': woman_base, 'crb': crb, 'protein': protein, 'fat': fat}

    # Eat-Fit 목적에 따른 계산
    if purpose == 'plus':
        user['cal'] *= 1.2
    elif purpose == 'lose':
        user['cal'] *= 0.8

    # 활동량에 따른 계산
    activity_levels = {1: 1.2, 2: 1.375, 3: 1.55, 4: 1.725, 5: 1.9}
    user['cal'] *= activity_levels[activity]
    user['cal'] = int(user['cal'])
    
    # 단위를 맞춰주기 위해 계산 결과를 조정
    user['crb'] = int(round(user['cal'] * crb / 4, 1))
    user['protein'] = int(round(user['cal'] * protein / 4,1))
    user['fat'] = int(round(user['cal'] * fat / 9,1))

    return user

# Ajax를 통해서 넘어온 유저 정보 기반 cal, crb, protein, fat 담아두는 기능.
def user(cal, crb, protein, fat):
    user = {'cal':cal, 'crb':crb,'protein':protein,'fat':fat}
    return user

In [3]:
# 밥과 면 종류에서 특정 값 내에 부합한 음식만 가져오기
def rice_noodle_rec(df,need_crb, need_cal, need_protein, need_fat):
    food_crb_lst = []
    mem_remain_crb_lst = []
    for i in range(len(df)):
        food_crb=df.iloc[i]['탄수화물(g)']
        food_cal=df.iloc[i]['열량(kcal)']
        food_protein=df.iloc[i]['단백질(g)']
        food_fat=df.iloc[i]['지방(g)']

        remain_user_crb = need_crb
        remain_user_cal = need_cal
        remain_user_protein = need_protein
        remain_user_fat = need_fat

        if (need_crb*0.60) <= food_crb <= (need_crb*0.85):
            food_crb_lst.append(df.iloc[i])
            remain_crb = int(round(remain_user_crb - food_crb,1))
            remain_cal = int(round(remain_user_cal - food_cal,1))
            remain_protein = int(round(remain_user_protein - food_protein,1))
            remain_fat = int(round(remain_user_fat - food_fat,1))

            mem_remain_crb_lst.append({'FOOD_RICE_SEQ':df.iloc[i]['FOOD_SEQ'] ,'remain_crb':remain_crb,
                                   'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})
    #print(user['crb']) # 이렇게 하면 마지막 나온 친구에서 crb가 빠짐
    food_crb_df = pd.DataFrame(food_crb_lst)
    mem_remain_crb_df = pd.DataFrame(mem_remain_crb_lst)
    
    # cal 마이너스인거 삭제하기
    i = 0
    while i < len(mem_remain_crb_df):
        if mem_remain_crb_df.iloc[i]['remain_cal']<0:
            mem_remain_crb_df = mem_remain_crb_df.drop(mem_remain_crb_df.index[i])
        else:
            i += 1
    
    # crb 마이너스 5 이상 삭제하기
    i = 0
    while i < len(mem_remain_crb_df):
        if mem_remain_crb_df.iloc[i]['remain_crb']< (-5):
            mem_remain_crb_df = mem_remain_crb_df.drop(mem_remain_crb_df.index[i])
        else:
            i += 1
            
    # fat 마이너스 5 이상 삭제하기
    i = 0
    while i < len(mem_remain_crb_df):
        if mem_remain_crb_df.iloc[i]['remain_fat']< (-5):
            mem_remain_crb_df = mem_remain_crb_df.drop(mem_remain_crb_df.index[i])
        else:
            i += 1
            
     # protein 마이너스 10 이상 삭제하기
    i = 0
    while i < len(mem_remain_crb_df):
        if mem_remain_crb_df.iloc[i]['remain_protein']< (-5):
            mem_remain_crb_df = mem_remain_crb_df.drop(mem_remain_crb_df.index[i])
        else:
            i += 1
    return mem_remain_crb_df

In [4]:
# 국 종류에서 특정 값 내에 부합한 음식만 가져오기
# df = mem_re_crb_df 고 df2 = soup_df입니다.
def soup_rec(df,df2, need_cal):
    food_soup_lst = []
    mem_remain_cal_lst = []
    for i in range(len(df2)):
        food_crb=df2.iloc[i]['탄수화물(g)']
        food_cal=df2.iloc[i]['열량(kcal)']
        food_protein=df2.iloc[i]['단백질(g)']
        food_fat=df2.iloc[i]['지방(g)']
        for j in range(len(df)):
            remain_user_crb = df.iloc[j]['remain_crb']
            remain_user_cal = df.iloc[j]['remain_cal']
            remain_user_protein = df.iloc[j]['remain_protein']
            remain_user_fat = df.iloc[j]['remain_fat']

            if (need_cal*0.1) <= food_cal <= (need_cal*0.2):
                food_soup_lst.append(df2.iloc[i])
                remain_crb = int(round(remain_user_crb - food_crb,1))
                remain_cal = int(round(remain_user_cal - food_cal,1))
                remain_protein = int(round(remain_user_protein - food_protein,1))
                remain_fat = int(round(remain_user_fat - food_fat,1))
                mem_remain_cal_lst.append({'FOOD_SOUP_SEQ':df2.iloc[i]['FOOD_SEQ'],'FOOD_RICE_SEQ':df.iloc[j]['FOOD_RICE_SEQ'] ,'remain_crb':remain_crb,
                                           'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})

    #print(user['crb']) # 이렇게 하면 마지막 나온 친구에서 crb가 빠짐
    food_soup_df = pd.DataFrame(food_soup_lst)
    mem_remain_cal_df = pd.DataFrame(mem_remain_cal_lst)
    
    # cal 마이너스인거 삭제하기
    i = 0
    while i < len(mem_remain_cal_df):
        if mem_remain_cal_df.iloc[i]['remain_cal']<0:
            mem_remain_cal_df = mem_remain_cal_df.drop(i)
        else:
            i += 1
    
    # crb 마이너스 5 인거 삭제하기
    i = 0
    while i < len(mem_remain_cal_df):
        if mem_remain_cal_df.iloc[i]['remain_crb']<(-5):
            mem_remain_cal_df = mem_remain_cal_df.drop(i)
        else:
            i += 1
            
    # fat 마이너스 5 인거 삭제하기
    i = 0
    while i < len(mem_remain_cal_df):
        if mem_remain_cal_df.iloc[i]['remain_fat']<(-5):
            mem_remain_cal_df = mem_remain_cal_df.drop(i)
        else:
            i += 1
            
    # protein 마이너스 15 인거 삭제하기
    i = 0
    while i < len(mem_remain_cal_df):
        if mem_remain_cal_df.iloc[i]['remain_protein']<(-15):
            mem_remain_cal_df = mem_remain_cal_df.drop(i)
        else:
            i += 1
            
    return mem_remain_cal_df

In [5]:
# 메인 반찬 종류에서 특정 값 내에 부합한 음식만 가져오기
# df = mem_re_cal_df 고 df2 = main_df입니다.
def main_rec(df,df2):
    food_main_lst = []
    mem_remain_fat_protein_lst = []
    for i in range(len(df2)):
        food_crb=df2.iloc[i]['탄수화물(g)']
        food_cal=df2.iloc[i]['열량(kcal)']
        food_protein=df2.iloc[i]['단백질(g)']
        food_fat=df2.iloc[i]['지방(g)']
        for j in range(len(df)):
            remain_user_crb = df.iloc[j]['remain_crb']
            remain_user_cal = df.iloc[j]['remain_cal']
            remain_user_protein = df.iloc[j]['remain_protein']
            remain_user_fat = df.iloc[j]['remain_fat']

            if (remain_user_protein*0.65) <= food_protein <= (remain_user_protein*0.85) or (remain_user_fat*0.65)<= food_fat <= (remain_user_fat*0.85):
                food_main_lst.append(df2.iloc[i])
                remain_crb = int(round(remain_user_crb - food_crb,1))
                remain_cal = int(round(remain_user_cal - food_cal,1))
                remain_protein = int(round(remain_user_protein - food_protein,1))
                remain_fat = int(round(remain_user_fat - food_fat,1))
                mem_remain_fat_protein_lst.append({'FOOD_MAIN_SEQ': df2.iloc[i]['FOOD_SEQ'],'FOOD_SOUP_SEQ':df.iloc[i]['FOOD_SOUP_SEQ'],'FOOD_RICE_SEQ':df.iloc[j]['FOOD_RICE_SEQ'] ,
                                                   'remain_crb':remain_crb,'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})

    food_main_df = pd.DataFrame(food_main_lst)
    mem_remain_fat_protein_df = pd.DataFrame(mem_remain_fat_protein_lst)

    # cal 마이너스인거 삭제하기
    i = 0
    while i < len(mem_remain_fat_protein_df):
        if mem_remain_fat_protein_df.iloc[i]['remain_cal']<0:
            mem_remain_fat_protein_df = mem_remain_cal_df.drop(i)
        else:
            i += 1
            
    # crb 마이너스5인거 삭제하기
    i = 0
    while i < len(mem_remain_fat_protein_df):
        if mem_remain_fat_protein_df.iloc[i]['remain_crb']<(-5):
            mem_remain_fat_protein_df = mem_remain_cal_df.drop(i)
        else:
            i += 1
            
    # fat 마이너스5인거 삭제하기
    i = 0
    while i < len(mem_remain_fat_protein_df):
        if mem_remain_fat_protein_df.iloc[i]['remain_fat']<(-5):
            mem_remain_fat_protein_df = mem_remain_cal_df.drop(i)
        else:
            i += 1
            
    # protein 마이너스15인거 삭제하기
    i = 0
    while i < len(mem_remain_fat_protein_df):
        if mem_remain_fat_protein_df.iloc[i]['remain_protein']<(-15):
            mem_remain_fat_protein_df = mem_remain_cal_df.drop(i)
        else:
            i += 1
            
    return mem_remain_fat_protein_df

In [6]:
# 서브 반찬 종류에서 특정 값 내에 부합한 음식만 가져오기
# df = mem_re_fat_pro_df 고 df2 = sub_df입니다.
def sub_rec(df,df2):
    food_sub_lst =[]
    mem_final_lst = []
    for i in range(len(df2)):
        food_crb=df2.iloc[i]['탄수화물(g)']
        food_cal=df2.iloc[i]['열량(kcal)']
        food_protein=df2.iloc[i]['단백질(g)']
        food_fat=df2.iloc[i]['지방(g)']
        for j in range(len(df)):
            remain_user_crb = df.iloc[j]['remain_crb']
            remain_user_cal = df.iloc[j]['remain_cal']
            remain_user_protein = df.iloc[j]['remain_protein']
            remain_user_fat = df.iloc[j]['remain_fat']

            if (remain_user_protein*0.70) <= food_protein <= remain_user_protein or (remain_user_fat*0.70)<= food_fat <= remain_user_fat or (remain_user_cal*0.7) <= food_cal <= remain_user_cal or (remain_user_crb*0.70) <= food_crb <= remain_user_crb:
                food_sub_lst.append(df2.iloc[i])
                remain_crb = int(round(remain_user_crb - food_crb,1))
                remain_cal = int(round(remain_user_cal - food_cal,1))
                remain_protein = int(round(remain_user_protein - food_protein,1))
                remain_fat = int(round(remain_user_fat - food_fat,1))
                mem_final_lst.append({'FOOD_RICE_SEQ':df.iloc[j]['FOOD_RICE_SEQ'] ,'FOOD_SOUP_SEQ':df.iloc[i]['FOOD_SOUP_SEQ'],'FOOD_MAIN_SEQ': df.iloc[i]['FOOD_MAIN_SEQ'],
                                      'FOOD_SUB_SEQ':df2.iloc[i]['FOOD_SEQ'],'remain_crb':remain_crb,'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})
    food_sub_df = pd.DataFrame(food_sub_lst)
    mem_final_df = pd.DataFrame(mem_final_lst)

    # cal 마이너스인거 삭제하기
    i = 0
    while i < len(mem_final_df):
        if mem_final_df.iloc[i]['remain_cal']<0:
            mem_final_df = mem_final_df.drop(mem_final_df.index[i])
        else:
            i += 1
            
    # fat 마이너스 5 이상 인거 삭제하기
    i = 0
    while i < len(mem_final_df):
        if mem_final_df.iloc[i]['remain_fat']<(-5):
            mem_final_df = mem_final_df.drop(mem_final_df.index[i])
        else:
            i += 1

    # crb 마이너스 5 이상 인거 삭제하기
    i = 0
    while i < len(mem_final_df):
        if mem_final_df.iloc[i]['remain_crb']<(-5):
            mem_final_df = mem_final_df.drop(mem_final_df.index[i])
        else:
            i += 1

    # protein 마이너스 15 이상 인거 삭제하기
    i = 0
    while i < len(mem_final_df):
        if mem_final_df.iloc[i]['remain_protein']<(-15):
            mem_final_df = mem_final_df.drop(mem_final_df.index[i])
        else:
            i += 1
            
    return mem_final_df

In [10]:
# 식단 추천 로직 만들기
# 얘는 아침밥 먹을 때 사용할 로직
def recommend_m_meal():
    
    user=user_PAL(72,176,25,'m',3,'maintain')
    
    df=open_excel()
    
    # 전체 df에서 한식, 일식만 가져옴
    kor_jap_df = df[df['한중일양'].isin(['한', '일'])]
    # 한,일식 df에서 밥,면 종류만 가져옴
    rice_noodle_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['밥','면'])]
    soup_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['국'])]
    main_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['메'])]
    sub_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['반'])]
    
    # 필요 영양성분 데이터
    need_crb=user['crb']/3
    need_cal = user['cal']/3
    need_protein=user['protein']/3
    need_fat = user['fat']/3

    mem_remain_crb_df = rice_noodle_rec(rice_noodle_df, need_crb, need_cal, need_protein, need_fat)
    
    mem_remain_cal_df = soup_rec(mem_remain_crb_df, soup_df, need_cal)
    
    mem_remain_fat_protein_df =main_rec(mem_remain_cal_df, main_df)
    
    mem_final_df =sub_rec(mem_remain_fat_protein_df, sub_df)
    
    return mem_final_df

In [8]:
# 식단 추천 로직 만들기
# 얘는  먹을 때 사용할 로직
def recommend_m_meal():
    
    user=user_PAL(72,176,25,'m',3,'maintain')
    
    df=open_excel()
    
    # 전체 df에서 한식, 일식만 가져옴
    kor_jap_df = df[df['한중일양'].isin(['한', '일'])]
    # 한,일식 df에서 밥,면 종류만 가져옴
    rice_noodle_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['밥','면'])]
    soup_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['국'])]
    main_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['메'])]
    sub_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['반'])]
    
    # 필요 영양성분 데이터
    need_crb=user['crb']/3
    need_cal = user['cal']/3
    need_protein=user['protein']/3
    need_fat = user['fat']/3

    mem_remain_crb_df = rice_noodle_rec(rice_noodle_df, need_crb, need_cal, need_protein, need_fat)
    
    mem_remain_cal_df = soup_rec(mem_remain_crb_df, soup_df)
    
    mem_remain_fat_protein_df =main_rec(mem_remain_cal_df, main_df)
    
    mem_final_df =sub_rec(mem_remain_fat_protein_df, sub_df)
    
    return mem_final_df

In [11]:
final_df = recommend_m_meal()
final_df

KeyError: '[200] not found in axis'